# Deploying our Application

### Introduction

Now that we have a running EC2 instance, the next step is to move the code from our local computer to our EC2 instance.  To do this, we'll need to install the required software like pip, copy the relevant local files to our machine, and boot up our flask application.  Let's get started.  

### Installing Required Software

To install the necessary software we need to access our machine via ssh.  

`ssh -i "flaskapp.pem" ec2-user@ec2-18-206-145-235.compute-1.amazonaws.com -vvv`

> We do this by referencing our pem file, and the EC2 endpoint.  The `-vvv` stands for verbose and provides a log of our files.

Ok, now it's time to install the necessary software.  On linux, we can do this with the `yum` command, which is an installer for linux.  We'll run the following:

* `sudo yum upgrade`

> Upgrade yum to the latest version.

Then we can use yum to install `python3`, `pip`, and `tmux`.

* `sudo yum install python3 python3-pip tmux`

Then, still logged into our EC2 machine, let's create a new folder which is where we'll place our `app`.

* `mkdir /home/ec2-user/app`

### Adding our Application

Next it's time to move the codebase to our EC2 machine.  We can do so by navigating to the parent folder of the code we want to copy.  And then copy our code to our ec2 machine with something like the following.

<img src="./scp_ip_address.png" width="100%">

The line above uses the `-r` flag to copy the contents of the folder to the target destination.  

> We specify the target destination with the following pattern `user@ec2_endpoint:target_folder`.  

Ok, now that the code has been moved over to our application, ssh back into our EC2 machine and install the packages specified in `requirements.txt`.  We change into our `app` directory and then run the following. 

* `pip3 install --user -r requirements.txt`

### Making Flask Visible

Next we can boot up our flask app, but it turns out we still won't be able to view this from AWS.  Let's go to the flask documentation to see why.

> If you run the server you will notice that the server is only accessible from your own computer, not from any other in the network. This is the default because in debugging mode a user of the application can execute arbitrary Python code on your computer.

> If you have the debugger disabled or trust the users on your network, you can make the server publicly available simply by adding --host=0.0.0.0 to the command line:

> `$ flask run --host=0.0.0.0`

> This tells your operating system to listen on all public IPs.

So if we change the host to `0.0.0.0`, it will listen for requests made to our public ip address.  We should also specify the port to line up to the port exposed on our EC2 machine: `port 80`.  We can set both of these with the following:

```python
app.run(host='0.0.0.0', port=80)
```

Ok, we're almost done.  In fact, we can boot up our flask app, by running the following:

`sudo python3 run.py`.

> If we do not run as sudo, we are likely to get a permission denied problem.

### Keep Listening with Tmux

There's just one problem.  How can we exit out of our EC2 shell without terminating the server that's running the flask app?  We need to use tmux to solve this.

We installed tmux above when we ran `sudo yum install tmux`.  And this will allow us to manage different sessions in our terminal.  We'll use this to create a new session with our flask app running, and then allow that session to keep our flask app running, while we detach from the session and exit out of the terminal.

We'll start by creating a new tmux session.

`tmux new -s mytestapp`

Then from inside of that session we again boot up our flask app.

`sudo python3 run.py`

Then we can detach from the tmux session by pressing `ctl-b` followed by `d`.

`ctl-b-d`

Now if we list the tmux sessions, we'll see that the session still exists, we are just currently detached from the session.

```bash
$ tmux ls
logout
Connection to 18.206.145.235 closed
```

Then we can safely exit from our EC2 instance, without shutting down our flask server.  And now, if we visit our public ip address, we'll see our application running.

<img src="./welcome-to-the-app.png" width="70%">

### Summary

In this lesson, we saw how we can move our deploy our flask application on our running EC2 instance.   We do so by connecting to our EC2 instance via ssh, and installing the requisite software.

`ssh -i "flaskapp.pem" ec2-user@ec2-18-206-145-235.compute-1.amazonaws.com -vvv`

* `sudo yum upgrade`
* `sudo yum install python3 python3-pip tmux`

Then we move over our code with scp and install our pip packages.

<img src="./scp-to-aws.png" width="80%">

* `sudo pip3 install -r requirements.txt`

We change our port and ip so that our app listens to public requests on the exposed port:
    
```python
app.run(host='0.0.0.0', port=80)
```

Then we use tmux to create a new session in our terminal, boot up our flask app, and then detach from the session, and exit the terminal.

`tmux new -s mytestapp`

`sudo pip3 run.py`

`ctl-b-d`

`exit`

### Resources

[Twilio Deploy Flask Application](https://www.twilio.com/blog/deploy-flask-python-app-aws)

[Docker Compose EC2](https://devops4solutions.com/docker-and-docker-compose-setup-on-aws-ec2-instance/)

[ECS Elastic Container Service](https://docs.aws.amazon.com/AmazonECS/latest/developerguide/ecs-cli-tutorial-ec2.html)

[Tmux Crash Course](https://thoughtbot.com/blog/a-tmux-crash-course)